In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import sys

In [3]:
#initializing slope lists to be filled
#the line that appends the slopes to these lists stays commented out, uncomment when ready to save slopes
#then comment out when finished
m_uvw1 = []
#m_uvw1.clear() #uncomment when need to clear 
m_uvw2 = []
#m_uvw2.clear()
m_uvm2 = []
#m_uvm2.clear()
m_U = []
#m_U.clear()
m_B = []
#m_B.clear()
m_V = [] 
#m_V.clear() 

def new_LinearR(file,uvw2start,uvw2end,uvm2start,uvm2end,uvw1start,uvw1end,ustart,uend,bstart,bend,vstart,vend): 
    global m_uvw2, m_uvm2, m_uvw1, m_U, m_B, m_V #telling python that these are global variables
    #file needs to be in quotes
    #portion that sets up the arrays to plot the light curve using file and filter / name
    data = open(file, 'r')
    filename = file.rsplit(".", 1)[0] #gets rid of .txt extension when naming plots
    # Reading the data in from the photometry file
    for line in data:
        if not line[0] == "#":
            continue
        fil, mjd, mag, magerr = np.loadtxt(data, dtype=str, usecols = (0,1,2,3), unpack=True) #unpack lets you assign columns names
    for m in range(len(mag)):
        if mag[m]=='NULL':
            mag[m]='nan'
    for me in range(len(magerr)):
        if magerr[me]=="NULL":
            magerr[me]='nan'
    mjd1 = [float(date) for date in mjd]
    mag1 = [float(magnitude) for magnitude in mag]
    magerr1 = [float(magnitudeerr) for magnitudeerr in magerr]
    filters1 = fil
    # I needed to get rid of the NULL values in the ...15.1.dat files, so the next several lines are to make sure the 
    # program doesn't shut down because of them
    filterslist = []
    mjdlist  = []
    maglist = []
    magerrlist = []
    for ii in range(len(filters1)):
        if not np.isnan(mag1[ii]):
            filterslist.append(filters1[ii])
            maglist.append(mag1[ii])
            mjdlist.append(mjd1[ii])
            magerrlist.append(magerr1[ii])
    filters = np.array(filterslist)
    mjd = np.array(mjdlist) #modified julian date
    mag = np.array(maglist) #magnitude
    magerr = np.array(magerrlist) #magnitude error
    
    # Initializing lists needed to plot the different filters separately
    #making empty lists so they are ready to be filled
    uvw2mjd = []
    uvw2mag = []
    uvw2magerr = []
    uvw2_mjd_lr = [] 
    uvw2_mag_lr = []
    uvm2mjd = []
    uvm2mag = []
    uvm2magerr = []
    uvm2_mjd_lr = [] 
    uvm2_mag_lr = []
    uvw1mjd = []
    uvw1mag = []
    uvw1magerr = []
    uvw1_mjd_lr = [] 
    uvw1_mag_lr = []
    umjd = []
    umag = []
    umagerr = []
    u_mjd_lr = [] 
    u_mag_lr = []
    bmjd = []
    bmag = []
    bmagerr = []
    b_mjd_lr = [] 
    b_mag_lr = []
    vmjd = []
    vmag = []
    vmagerr = []
    v_mjd_lr = [] 
    v_mag_lr = [] 
    
    # breaking up the filters, mjd, mag, and magerr arrays into separate arrays to make plotting easier
    for i in range(len(filters)):
        if filters[i] == 'UVW2':
            uvw2mjd.append(mjd[i]) #i means which row
            uvw2mag.append(mag[i])
            uvw2magerr.append(magerr[i])
        if filters[i] == 'UVM2':
            uvm2mjd.append(mjd[i])
            uvm2mag.append(mag[i])
            uvm2magerr.append(magerr[i])
        if filters[i] == 'UVW1':
            uvw1mjd.append(mjd[i])
            uvw1mag.append(mag[i])
            uvw1magerr.append(magerr[i])
        if filters[i] == 'U':
            umjd.append(mjd[i])
            umag.append(mag[i])
            umagerr.append(magerr[i])
        if filters[i] == 'B':
            bmjd.append(mjd[i])
            bmag.append(mag[i])
            bmagerr.append(magerr[i])
        if filters[i] == 'V':
            vmjd.append(mjd[i])
            vmag.append(mag[i])
            vmagerr.append(magerr[i])
            
    if uvw2start and uvw2end != '0':
        for i in range(len(uvw2mjd)):
                if uvw2mjd[i] > uvw2start and uvw2mjd[i] < uvw2end:
                    uvw2_mjd_lr.append(uvw2mjd[i])
                    uvw2_mag_lr.append(uvw2mag[i])
        #uvw2 LR
        X1 = np.array(uvw2_mjd_lr)
        Y1 = np.array(uvw2_mag_lr)
        x1 = X1.reshape(-1, 1) 
        y1 = Y1.reshape(-1, 1)  
        linear_regressor = LinearRegression()  
        linear_regressor.fit(x1,y1)  
        y1_pred = linear_regressor.predict(x1)

        if len(y1_pred) >= 2 and len(x1) >= 2:
            uvw2_slope = float((y1_pred[-1] - y1_pred[0]) / (x1[-1] - x1[0]))
        else:
            uvw2_slope = float('nan')
        #m_uvw2.append(uvw2_slope)

        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)  
        if len(uvw2mag) > 0:
            ax.scatter(uvw2mjd, uvw2mag, linewidth=1,marker='o', facecolors='none', edgecolors='r', s=30, label='uvw2', color = 'r')
            #ax.errorbar(uvw2mjd, uvw2mag, yerr=uvw2magerr, elinewidth=2, capthick = 2, color = 'r') #red
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(min(uvw2mjd))-1, math.ceil(np.amax(uvw2mjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim_1 = plt.xlim(uvw2mjd[0]-1,uvw2mjd[-1]+1)
            ylim_1 = plt.ylim(uvw2mag[0]-2,uvw2mag[-1]+2)
            plt.gca().invert_yaxis() # Invert the y-axis
            plt.plot(x1,y1_pred,color='black')
            # Define text position as a percentage of the x and y limits
            #x1_pos = xlim_1[0] + 0.6 * (xlim_1[1] - xlim_1[0])  # 60% across the x-axis
            #y1_pos = ylim_1[0] + 0.8 * (ylim_1[1] - ylim_1[0])  # 80% across the y-axis
            #plt.text(x1_pos,y1_pos, f"m = {uvw2_slope:.3f}", fontsize = 12, color = 'red') 
            plt.title(f"Linear Regression and Light Curve for {filename} UVW2")
            plt.savefig(f"{filename}_UVW2.png", dpi=300, bbox_inches="tight")
            plt.show()
            
    if uvm2start and uvm2end != '0':
        for i in range(len(uvm2mjd)):
                if uvm2mjd[i] > uvm2start and uvm2mjd[i] < uvm2end:
                    uvm2_mjd_lr.append(uvm2mjd[i])
                    uvm2_mag_lr.append(uvm2mag[i])
        #uvm2 LR
        X2 = np.array(uvm2_mjd_lr) 
        Y2 = np.array(uvm2_mag_lr)
        x2 = X2.reshape(-1, 1)  
        y2 = Y2.reshape(-1, 1)  
        linear_regressor = LinearRegression()  
        linear_regressor.fit(x2,y2)  
        y2_pred = linear_regressor.predict(x2)
        

        
        if len(y2_pred) >= 2 and len(x2) >= 2:
            uvm2_slope = float((y2_pred[-1] - y2_pred[0]) / (x2[-1] - x2[0]))
        else:
            uvm2_slope = float('nan')
            
        #m_uvm2.append(uvm2_slope)
    
        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if len(uvm2mag) > 0:
            ax.scatter(uvm2mjd, uvm2mag, linewidth=1,marker='o',facecolors='none', edgecolors='m', s=30, label='uvm2', color= 'm')
            #ax.errorbar(uvm2mjd, uvm2mag, yerr=uvm2magerr, elinewidth=2, capthick = 2, color='m') #magenta
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(mjd.min())-1, math.ceil(np.amax(uvw2mjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim_2 = plt.xlim(uvm2mjd[0]-1,uvm2mjd[-1]+1)
            ylim_2 = plt.ylim(uvm2mag[0]-2,uvm2mag[-1]+2)
            plt.gca().invert_yaxis() # Invert the y-axis
            # Define text position as a percentage of the x and y limits
            #x2_pos = xlim_2[0] + 0.6 * (xlim_2[1] - xlim_2[0])  # 60% across the x-axis
            #y2_pos = ylim_2[0] + 0.8 * (ylim_2[1] - ylim_2[0])  # 80% across the y-axis
            #plt.text(x2_pos,y2_pos, f"m = {uvm2_slope:.3f}", fontsize = 12, color = 'red') 
            plt.plot(x2,y2_pred,color='black')
            plt.title(f"Linear Regression and Light Curve for {filename} UVM2")
            plt.savefig(f"{filename}_UVM2.png", dpi=300, bbox_inches="tight")
            plt.show()
            
    if uvw1start and uvw1end != '0':
        for i in range(len(uvw1mjd)):
            if uvw1mjd[i] > uvw1start and uvw1mjd[i] < uvw1end:
                uvw1_mjd_lr.append(uvw1mjd[i])
                uvw1_mag_lr.append(uvw1mag[i])
        
        #uvw1 LR 
        X = np.array(uvw1_mjd_lr) 
        Y = np.array(uvw1_mag_lr)
        x = X.reshape(-1, 1)  # values converts it into a numpy array
        y = Y.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
        linear_regressor = LinearRegression()  # create object for the class
        linear_regressor.fit(x,y)  # perform linear regression
        y_pred = linear_regressor.predict(x)  # make predictions
        
        if len(y_pred) >= 2 and len(x) >= 2:
            uvw1_slope = float((y_pred[-1] - y_pred[0]) / (x[-1] - x[0]))
        else:
            uvw1_slope = float('nan')
        #m_uvw1.append(uvw1_slope)
       
        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if len(uvw1mag) > 0:
            ax.scatter(uvw1mjd, uvw1mag, linewidth=1, marker='o', facecolors='none', edgecolors='k', s=30, label='uvw1', color= 'k')
            #ax.errorbar(uvw1mjd, uvw1mag, yerr=uvw1magerr, elinewidth=2, capthick = 2, color='k') # black
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(mjd.min())-1, math.ceil(np.amax(uvw1mjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim = plt.xlim(uvw1mjd[0]-1,uvw1mjd[-1]+1) #had to expand x-axis b/c data wasnt graphing right
            ylim = plt.ylim(uvw1mag[0]-2,uvw1mag[-1]+2)
            plt.gca().invert_yaxis() # Invert the y-axis
            # Define text position as a percentage of the x and y limits
            #x_pos = xlim[0] + 0.6 * (xlim[1] - xlim[0])  # 60% across the x-axis
            #y_pos = ylim[0] + 0.8 * (ylim[1] - ylim[0])  # 80% across the y-axis
            #plt.text(x_pos,y_pos, f"m = {uvw1_slope:.3f}", fontsize = 12, color = 'red') 
            plt.plot(x,y_pred,color='red')
            plt.title(f"Linear Regression and Light Curve for {filename} UVW1")
            plt.savefig(f"{filename}_UVW1.png", dpi=300, bbox_inches="tight")
            plt.show() 
            
    if ustart and uend != '0':
        for i in range(len(umjd)):
            if umjd[i] > ustart and umjd[i] < uend:
                u_mjd_lr.append(umjd[i])
                u_mag_lr.append(umag[i])
        
        #U LR
        X3 = np.array(u_mjd_lr) 
        Y3 = np.array(u_mag_lr)
        x3 = X3.reshape(-1, 1)  
        y3 = Y3.reshape(-1, 1)  
        linear_regressor = LinearRegression()  
        linear_regressor.fit(x3,y3)  
        y3_pred = linear_regressor.predict(x3)

        if len(y3_pred) >= 2 and len(x3) >= 2:
            u_slope = float((y3_pred[-1] - y3_pred[0]) / (x3[-1] - x3[0]))
        else:
            u_slope = float('nan')
            
        #m_U.append(u_slope)
        
        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if len(umag) > 0:
            ax.scatter(umjd, umag, linewidth=1, marker='o', facecolors='none', edgecolors='c', s=30, label='uvot u', color= 'c')
            #ax.errorbar(umjd, umag, yerr=umagerr, elinewidth=2, capthick = 2, color='c') #cyan
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(min(umjd))-1, math.ceil(np.amax(umjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim_3 = plt.xlim(umjd[0]-1,umjd[-1]+1)
            ylim_3 = plt.ylim(umag[0]-2,umag[-1]+2)
            # Define text position as a percentage of the x and y limits
            x3_pos = xlim_3[0] + 0.6 * (xlim_3[1] - xlim_3[0])  # 60% across the x-axis
            y3_pos = ylim_3[0] + 0.8 * (ylim_3[1] - ylim_3[0])  # 80% across the y-axis
            #plt.text(x3_pos,y3_pos, f"m = {u_slope:.3f}", fontsize = 12, color = 'red') 
            plt.gca().invert_yaxis() # Invert the y-axis
            plt.plot(x3,y3_pred,color='black')
            plt.title(f"Linear Regression and Light Curve for {filename} U")
            plt.savefig(f"{filename}_U.png", dpi=300, bbox_inches="tight")
            plt.show()
            
    if bstart and bend != '0':
        for i in range(len(bmjd)):
            if bmjd[i] > bstart and bmjd[i] < bend:
                b_mjd_lr.append(bmjd[i])
                b_mag_lr.append(bmag[i])

        #B LR
        X5 =np.array(b_mjd_lr) 
        Y5 =np.array(b_mag_lr)
        x5 = X5.reshape(-1, 1)  
        y5 = Y5.reshape(-1, 1) 
        linear_regressor = LinearRegression()  
        linear_regressor.fit(x5,y5)  
        y5_pred = linear_regressor.predict(x5)  

        if len(y5_pred) >= 2 and len(x5) >= 2:
            b_slope = float((y5_pred[-1] - y5_pred[0]) / (x5[-1] - x5[0]))
        else:
            b_slope = float('nan')
            
        #m_B.append(b_slope)
        
        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if len(bmag) > 0:
            ax.scatter(bmjd, bmag, linewidth=1,marker='o',facecolors='none', edgecolors='b', s=50, label='uvot b', color= 'b')
            #ax.errorbar(bmjd, bmag, yerr=bmagerr, elinewidth=2, capthick = 2, color='b') #blue
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(min(bmjd))-1, math.ceil(np.amax(bmjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim_5 = plt.xlim(bmjd[0]-1,bmjd[-1]+1)
            ylim_5 = plt.ylim(bmag[0]-2,bmag[-1]+2)
            # Define text position as a percentage of the x and y limits
            x5_pos = xlim_5[0] + 0.6 * (xlim_5[1] - xlim_5[0])  # 60% across the x-axis
            y5_pos = ylim_5[0] + 0.8 * (ylim_5[1] - ylim_5[0])  # 80% across the y-axis
            #plt.text(x5_pos,y5_pos, f"m = {b_slope:.3f}", fontsize = 12, color = 'red') 
            plt.gca().invert_yaxis() # Invert the y-axis
            plt.plot(x5,y5_pred,color='black')
            plt.title(f"Linear Regression and Light Curve for {filename} B")
            plt.savefig(f"{filename}_B.png", dpi=300, bbox_inches="tight")
            plt.show()
            
    if vstart and vend != '0':
        for i in range(len(vmjd)):
            if vmjd[i] > vstart and vmjd[i] < vend:
                v_mjd_lr.append(vmjd[i])
                v_mag_lr.append(vmag[i])
        #V LR
        X4 = np.array(v_mjd_lr) 
        Y4 = np.array(v_mag_lr)
        x4 = X4.reshape(-1, 1)  
        y4 = Y4.reshape(-1, 1)  
        linear_regressor = LinearRegression()  
        linear_regressor.fit(x4,y4)  
        y4_pred = linear_regressor.predict(x4) 

        if len(y4_pred) >= 2 and len(x4) >= 2:
            v_slope = float((y4_pred[-1] - y4_pred[0]) / (x4[-1] - x4[0]))
        else:
            v_slope = float('nan')
            
        #m_V.append(v_slope) 
        
        
        #plotting code
        plt.ion() #turns on interactive plotting
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if len(vmag) > 0:
            ax.scatter(vmjd, vmag, linewidth=1,marker='o',facecolors='none', edgecolors='g', s=30, label='uvot v', color= 'g')
            #ax.errorbar(vmjd, vmag, yerr=vmagerr, elinewidth=2, capthick = 2, color='g') #green
            #ax.legend(bbox_to_anchor=(1.32,0.83))
            ax.set_xlabel('Modified Julian Date')
            ax.set_ylabel('Observed Magnitude')
            ax.axis([int(min(vmjd))-1, math.ceil(np.amax(vmjd))+1,math.ceil(np.amax(mag)+np.amax(magerr)), int(mag.min())-0.25])
            xlim_4 = plt.xlim(vmjd[0]-1,vmjd[-1]+1)
            ylim_4 = plt.ylim(vmag[0]-2,vmag[-1]+2)
            # Define text position as a percentage of the x and y limits
            x4_pos = xlim_4[0] + 0.6 * (xlim_4[1] - xlim_4[0])  # 60% across the x-axis
            y4_pos = ylim_4[0] + 0.8 * (ylim_4[1] - ylim_4[0])  # 80% across the y-axis
            #plt.text(x4_pos,y4_pos, f"m = {v_slope:.3f}", fontsize = 12, color = 'red') 
            plt.gca().invert_yaxis() # Invert the y-axis
            plt.plot(x4,y4_pred,color='black')
            plt.title(f"Linear Regression and Light Curve for {filename} V")
            plt.savefig(f"{filename}_V.png", dpi=300, bbox_inches="tight")
            plt.show()  
            
    #printing all the slopes for each line and rounding to 3 decimal places
    if uvw2start and uvw2end != '0':
        print(f"UVW2: {uvw2_slope:.3f}") 
    if uvm2start and uvm2end != '0':
        print(f"UVM2: {uvm2_slope:.3f}")
    if uvw1start and uvw1end != '0':
        print(f"UVW1: {uvw1_slope:.3f}")
    if ustart and uend != '0':
        print(f"U: {u_slope:.3f}")
    if vstart and vend != '0':
        print(f"V: {v_slope:.3f}") 
    if bstart and bend != '0':
        print(f"B: {b_slope:.3f}")    